# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

- importing essential packages like `Pandas` and `Cassandra`

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)                                     # Checking the file path

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

- Creating the cluster and connecting to the cluster
- The ip address `127.0.0.1` is used to connect to the clusture

In [ ]:
# Connection to a Cassandra instance on your local machine 
from cassandra.cluster import Cluster
cluster = Cluster()

# Making 
try: 
    cluster = Cluster(['127.0.0.1']) 
    session = cluster.connect()
except Exception as e:
    print(e)
# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

- Here a keyspace name `Udacity` is created.

In [ ]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

- A connection to the keyspace named Udacity is made.

In [ ]:
# Seting up KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Writing `first` query

- A table name `song_info_by_session` is created
- Here `sessionid` and  `iteminsession` are set as composite primary keys
- The first query to get the artist, song title and song's length in the music app history that was heard during \
  sessionId = 338, and itemInSession = 4 is written as follows.

In [ ]:
## Query 1:  Geting the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_info_by_session"
query = query + "(sessionid int, iteminsession int, artist varchar, song varchar, first_name text, length double, PRIMARY KEY (sessionid, iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                    

### Extracting data
- Extracting data from `CSV` file.
- Insert statement is used to insert data into table `song_info_by_session`.

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query = "INSERT INTO song_info_by_session (sessionid, iteminsession, artist, song, first_name, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s)"
        ## Assigning which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        ## The column numbers are added as per the event_datafile_new.csv
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], line[1], float(line[5]))) 
        

#### A SELECT to verify that the data have been inserted into each table

### Select statement to get first query.
- A select statement is used to query the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession = 4 from the music_app_history table
- Only the required data that is `artist`, `song`, `first_name`, and `length` is printed below as a result of the first query.


In [ ]:
## implementing the SELECT statement to verify the data was entered into the table
query = "select artist, song, first_name, length from song_info_by_session WHERE sessionid = 338 AND iteminsession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.length) 

### REPEATING THE ABOVE PROCESS FOR EACH OF THE REMANING THREE QUERIES.

### Writing the `second` query

- A table called `artist_info_by_session` is created as per the data requirement.
- Here `sessionid` and  `userid` are set as composite keys and `inteminsession`, is set as clustering columns of the primary key
- For the second query, the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 is required.
- A new query name `query1` is created and similar steps as followed in step one are implemented. 

In [ ]:
## Query 2: getting the following data: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query1 = "CREATE TABLE IF NOT EXISTS artist_info_by_session"
query1 = query1 + "(sessionid int, userid int, iteminsession int, first_name text, last_name text, artist varchar, song varchar, PRIMARY KEY ((sessionid, userid), iteminsession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)    

                    

### Extracting data for `second query`

- Extracting data from the same`event_datafile_new.csv` file.
- Insert statement is used to insert data into table `artist_info_by_session`.


In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query1 = "INSERT INTO artist_info_by_session (sessionid, userid, iteminsession, first_name, last_name, artist, song)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assigning column element for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query1, (int(line[8]), int(line[10]), int(line[3]), line[1], line[4], line[0], line[9]))

### Select statement to get `second` query.
- A select statement is used to query name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
- Only the required data that is `artist`, `song`, `first_name`, `last_name` and `iteminsession` is printed below as a result of the second query.


In [ ]:
## implementing the SELECT statement to verify the data was entered into the table
query1 = "select artist, song, first_name, last_name, iteminsession from artist_info_by_session WHERE sessionid = 182 AND userid = 10"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.first_name, row.last_name, row.iteminsession) 

### Third Query

### Writing the `Third` query

- A table called `user_song_history` is created as per the data requirement.
- Here `song` and `userid` column is set as composite keys of the primary key
- For the third query, every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' is querried.
- A new query name `query2` is created and similar steps as followed in step one are implemented. 

In [ ]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query2 = "CREATE TABLE IF NOT EXISTS user_song_history"
query2 = query2 + "(song varchar, userid int, first_name text, last_name text, PRIMARY KEY (song, userid))"
try:
    session.execute(query2)
except Exception as e:
    print(e)  


### Extracting data for `Third query`

- Extracting data from the same`event_datafile_new.csv` file.
- Insert statement is used to insert data into table `user_song_history`.


In [ ]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assigning the INSERT statements into the `query` variable
        query2 = "INSERT INTO user_song_history (song, userid, first_name, last_name)"
        query2 = query2 + "VALUES (%s, %s, %s, %s)"
        ## To Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query2, (line[9], int(line[10]), line[1], line[4]))

### Select statement to get `third` query.
- A select statement is used to get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
- Only the required data that is `song`, `first_name`, and `last_name` is printed below as a result of the second query.


In [ ]:
#implementing select statement for third query
query2 = "select song, first_name, last_name from user_song_history WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.song, row.first_name, row.last_name) 

### Drop the tables before closing out the sessions

- The tables `song_info_by_session`, `artist_info_by_session` and `user_song_history` are dropped after the queries are exucuted.

In [ ]:
## Droping the song_info_by_session table before closing out the sessions
query = "drop table song_info_by_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [ ]:
## Droping the artist_info_by_session table before closing out the sessions
query1 = "drop table artist_info_by_session"
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

In [ ]:
## Droping the user_song_history table before closing out the sessions
query2 = "drop table user_song_history"
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()